In [ ]:
import matplotlib.pyplot as plt
import random
import numpy as np
from collections import defaultdict
import math
from scipy import stats

In [ ]:
from lib.moments import init_moment_k, init_moment_k_interval, cent_moment_k, cent_moment_k_interval
from lib.sample_creation import get_sample

# **ЛР 3.2**